Import required libraries

In [11]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier as RFC

#pandas data object function
obj_fn = lambda x : pd.DataFrame(x)

Load the Data

In [136]:
movies = pd.read_csv('data/movies.csv')
ratings = pd.read_csv('data/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Check for the mean rating on each movie

In [137]:
movies_rating_data = obj_fn(ratings.groupby('movieId')['rating'].mean())
movies_rating_data.head()

,rating
movieId,
1,3.920930
2,3.431818
3,3.259615
4,2.357143
5,3.071429


Check the popularity of each movie among users

In [138]:
movies_rating_data['count'] = obj_fn(ratings.groupby('movieId')['userId'].count())
movies_rating_data.head()

,rating,count
movieId,,
1,3.920930,215
2,3.431818,110
3,3.259615,52
4,2.357143,7
5,3.071429,49


Build user-item utility matrix ,using pivot_table() function of pandas

In [139]:
crosstab = pd.pivot_table(data = ratings,values = 'rating',index = 'userId',columns = 'movieId')
crosstab.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Users rated less movies hence null values in crosstab<br>
View movieId = 260

In [140]:
movies.loc[movies['movieId'] == 260]

,movieId,title,genres
224,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi


This movie is Star Wars IV. Create a variable and remove null values

In [149]:
star_wars = crosstab[260]
star_wars = star_wars[star_wars >= 0]
star_wars

userId
1      5.0
4      5.0
7      5.0
15     5.0
16     3.0
      ... 
605    5.0
606    4.5
607    3.0
608    3.5
610    5.0
Name: 260, Length: 251, dtype: float64

Now the objective is to find similar movies. We will understand this by finding
movies similar to Star Wars.

In [179]:
similar = crosstab.corrwith(star_wars,drop = True)
similar_dframe = pd.DataFrame(similar , columns = ['PearsonR'])

C:\Users\admin\anaconda3\lib\site-packages\numpy\lib\function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\admin\anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
C:\Users\admin\anaconda3\lib\site-packages\numpy\lib\function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\admin\anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
C:\Users\admin\anaconda3\lib\site-packages\numpy\lib\function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\admin\anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
C:\Users\admin\a

In [180]:
similar_dframe.dropna(inplace = True)

In [181]:
similar_dframe.head()

,PearsonR
movieId,
1,0.119131
2,0.183448
3,0.175559
5,-0.090036
6,0.141549


Above code shows each movie id and their correlation with star_wars. This is not
very useful at this moment.
We also want to check for popular movies only which are rated by atleast 50 users.
To do this let's join our frame with rating

In [182]:
similar_dframe_summary = similar_dframe.join(movies_rating_data['count'])
similar_dframe_summary = similar_dframe_summary[similar_dframe_summary['count'] >= 50].sort_values('PearsonR',ascending = False).head()
similar_dframe_summary

,PearsonR,count
movieId,,
260,1.000000,251
1196,0.777970,211
1210,0.734230,196
457,0.482078,190
63082,0.479859,71


Get top 5 similar movies

In [183]:
top_5_similar = similar_dframe_summary.head().index.values.tolist()
top_5_similar

[260, 1196, 1210, 457, 63082]

Print top 10 similar movies to Star wars

In [184]:
movies.loc[movies['movieId'].isin(top_5_similar)]

,movieId,title,genres
224,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi
398,457,"Fugitive, The (1993)",Thriller
898,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi
911,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Sci-Fi
6885,63082,Slumdog Millionaire (2008),Crime|Drama|Romance


ImportError: cannot import name 'evaluate' from 'surprise' (C:\Users\admin\anaconda3\lib\site-packages\surprise\__init__.py)